In [ ]:
pip install pandas numpy scikit-learn seaborn matplotlib xgboost lightgbm streamlit surprise


Step 1: Set Up Your Environment


In [15]:
import pandas as pd

# Load the Excel file and print sheet names
file_path = "/content/Tourism Dataset (1).xlsx"
xls = pd.ExcelFile(file_path)
print("Available Sheets:", xls.sheet_names)


Available Sheets: ['User', 'Type', 'Transaction', 'Region', 'Mode ', 'Item', 'Country', 'Continent', 'City']


In [16]:
# Step 1: Merge transaction, user, item, city, type, continent
d = transaction.merge(user, on="UserId", how="left") \
                .merge(item, on="AttractionId", how="left") \
                .merge(city, left_on="AttractionCityId", right_on="CityId", how="left", suffixes=('', '_City')) \
                .merge(type_df, on="AttractionTypeId", how="left") \
                .merge(continent, on="ContinentId", how="left")

# Step 2: Merge country and preserve RegionId safely
country = country.rename(columns={"RegionId": "CountryRegionId"})  # avoid conflict
d = d.merge(country, on="CountryId", how="left")

# Step 3: Merge region using the renamed key
region = region.rename(columns={"RegionId": "CountryRegionId"})  # match with above
d = d.merge(region, on="CountryRegionId", how="left")

# Final cleaning
d.dropna(inplace=True)

# Show structure
print("✅ Final Merged DataFrame Shape:", df.shape)
print(d.columns.tolist())
print(d.head())


NameError: name 'transaction' is not defined

In [14]:
import pandas as pd

# Load the Excel file and print sheet names
file_path = "tf.xlsx"
xls = pd.ExcelFile(file_path)
print("Available Sheets:", xls.sheet_names)

# Load data from each sheet into a DataFrame
try:
    transaction = xls.parse('transaction') # Assuming a sheet named 'transaction'
    user = xls.parse('user')             # Assuming a sheet named 'user'
    item = xls.parse('item')             # Assuming a sheet named 'item'
    city = xls.parse('city')             # Assuming a sheet named 'city'
    # Adjust the sheet name based on the actual name in your Excel file
    type_df = xls.parse('type')          # Assuming a sheet named 'type' or similar
    continent = xls.parse('continent')   # Assuming a sheet named 'continent'
    country = xls.parse('country')       # Assuming a sheet named 'country'
    region = xls.parse('region')         # Assuming a sheet named 'region'

    print("✅ DataFrames loaded from Excel sheets.")

except Exception as e:
    print(f"❌ Error loading data from Excel sheets: {e}")
    print("Please ensure the sheet names in 'tf.xlsx' match the code.")

Available Sheets: ['User', 'Type', 'Transaction', 'Region', 'Mode ', 'Item', 'Country', 'Continent', 'City']
❌ Error loading data from Excel sheets: Worksheet named 'transaction' not found
Please ensure the sheet names in 'tf.xlsx' match the code.


1. Regression Model — Predict Rating



In [10]:
from sklearn.preprocessing import LabelEncoder

X_encoded = X.copy()
for col in ['ContinentId_x', 'RegionId', 'CountryId', 'CityId', 'AttractionTypeId']:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))


NameError: name 'X' is not defined

In [ ]:
reg_model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)


In [ ]:
sample_frac = 0.2  # Use 20% of data for faster iteration
X_sample = X_encoded.sample(frac=sample_frac, random_state=42)
y_sample = y.loc[X_sample.index]

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)


In [ ]:
import lightgbm as lgb

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
}

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

lgb_model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)


In [ ]:
lgb_model.save_model('lgbm_rating_model.txt')
print("✅ LightGBM model saved as 'lgbm_rating_model.txt'")


In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("📈 R2 Score:", round(r2, 3))


In [ ]:
import numpy as np

X = df[features_reg].copy()

# Cyclical encoding for month
X['VisitMonth_sin'] = np.sin(2 * np.pi * X['VisitMonth'] / 12)
X['VisitMonth_cos'] = np.cos(2 * np.pi * X['VisitMonth'] / 12)

# Cyclical encoding for year (assuming VisitYear covers several years)
max_year = X['VisitYear'].max()
min_year = X['VisitYear'].min()
X['VisitYear_sin'] = np.sin(2 * np.pi * (X['VisitYear'] - min_year) / (max_year - min_year))
X['VisitYear_cos'] = np.cos(2 * np.pi * (X['VisitYear'] - min_year) / (max_year - min_year))

# Drop original VisitMonth and VisitYear if you want
X = X.drop(['VisitMonth', 'VisitYear'], axis=1)


In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in ['ContinentId_x', 'RegionId', 'CountryId', 'CityId', 'AttractionTypeId']:
    X[col] = X[col].astype(str)  # convert to string just in case
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmses = []
r2s = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

   model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(0)]  # suppress output
)

    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = r2_score(y_val, y_pred)
    rmses.append(rmse)
    r2s.append(r2)

print(f"Average RMSE: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
print(f"Average R2 Score: {np.mean(r2s):.3f} ± {np.std(r2s):.3f}")


In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Prepare features with cyclical encoding
X = df[features_reg].copy()

# Cyclical encoding for month
X['VisitMonth_sin'] = np.sin(2 * np.pi * X['VisitMonth'] / 12)
X['VisitMonth_cos'] = np.cos(2 * np.pi * X['VisitMonth'] / 12)

# Cyclical encoding for year
max_year = X['VisitYear'].max()
min_year = X['VisitYear'].min()
X['VisitYear_sin'] = np.sin(2 * np.pi * (X['VisitYear'] - min_year) / (max_year - min_year))
X['VisitYear_cos'] = np.cos(2 * np.pi * (X['VisitYear'] - min_year) / (max_year - min_year))

# Drop original VisitMonth and VisitYear
X = X.drop(['VisitMonth', 'VisitYear'], axis=1)

# Label encode categorical features
for col in ['ContinentId_x', 'RegionId', 'CountryId', 'CityId', 'AttractionTypeId']:
    X[col] = X[col].astype(str)
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

y = df[target]

# Set up 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmses = []
r2s = []

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
}

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(0)]
    )

    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = r2_score(y_val, y_pred)
    rmses.append(rmse)
    r2s.append(r2)

print(f"Average RMSE: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
print(f"Average R2 Score: {np.mean(r2s):.3f} ± {np.std(r2s):.3f}")


In [ ]:
pip install imbalanced-learn


In [ ]:
pip install xgboost imbalanced-learn scikit-learn


In [ ]:
pip install --upgrade xgboost


In [ ]:
import xgboost
print(xgboost.__version__)  # Should print 1.3.0 or higher


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# -----------------------------
# Data Preparation
# -----------------------------
features_clf = [
    'VisitYear', 'VisitMonth', 'ContinentId_x', 'RegionId',
    'CountryId', 'CityId', 'AttractionTypeId'
]
target_clf = 'VisitMode'

X = df[features_clf].copy()
y = df[target_clf]

# Cyclical encoding for time-based features
X['VisitMonth_sin'] = np.sin(2 * np.pi * X['VisitMonth'] / 12)
X['VisitMonth_cos'] = np.cos(2 * np.pi * X['VisitMonth'] / 12)
X['VisitYear_sin'] = np.sin(2 * np.pi * (X['VisitYear'] - X['VisitYear'].min()) / (X['VisitYear'].max() - X['VisitYear'].min()))
X['VisitYear_cos'] = np.cos(2 * np.pi * (X['VisitYear'] - X['VisitYear'].min()) / (X['VisitYear'].max() - X['VisitYear'].min()))
X.drop(['VisitMonth', 'VisitYear'], axis=1, inplace=True)

# Label encode categorical features
for col in ['ContinentId_x', 'RegionId', 'CountryId', 'CityId', 'AttractionTypeId']:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Label encode target
y_le = LabelEncoder()
y_encoded = y_le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -----------------------------
# Train XGBoost Model
# -----------------------------
model = XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_encoded)),
    learning_rate=0.05,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    random_state=42
)

# Train with early stopping
model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    eval_metric='mlogloss',
    early_stopping_rounds=10
)

# -----------------------------
# Evaluate the Model
# -----------------------------
y_pred = model.predict(X_test)

print(f"\n✅ Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=[str(cls) for cls in y_le.classes_]))


In [ ]:
model = XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_encoded)),
    learning_rate=0.05,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    random_state=42,
    eval_metric='mlogloss'  # <- here instead of fit()
)

model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=10
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# -----------------------------
# Data Preparation
# -----------------------------
features_clf = [
    'VisitYear', 'VisitMonth', 'ContinentId_x', 'RegionId',
    'CountryId', 'CityId', 'AttractionTypeId'
]
target_clf = 'VisitMode'

X = df[features_clf].copy()
y = df[target_clf]

# Cyclical encoding for time-based features
X['VisitMonth_sin'] = np.sin(2 * np.pi * X['VisitMonth'] / 12)
X['VisitMonth_cos'] = np.cos(2 * np.pi * X['VisitMonth'] / 12)
X['VisitYear_sin'] = np.sin(
    2 * np.pi * (X['VisitYear'] - X['VisitYear'].min()) /
    (X['VisitYear'].max() - X['VisitYear'].min())
)
X['VisitYear_cos'] = np.cos(
    2 * np.pi * (X['VisitYear'] - X['VisitYear'].min()) /
    (X['VisitYear'].max() - X['VisitYear'].min())
)
X.drop(['VisitMonth', 'VisitYear'], axis=1, inplace=True)

# Label encode categorical features
for col in ['ContinentId_x', 'RegionId', 'CountryId', 'CityId', 'AttractionTypeId']:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Label encode target
y_le = LabelEncoder()
y_encoded = y_le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -----------------------------
# Train XGBoost Model
# -----------------------------
model = XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_encoded)),
    learning_rate=0.05,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    random_state=42,
    eval_metric='mlogloss'  # you can keep this if supported
)

model.fit(
    X_train_res,
    y_train_res,
    eval_set=[(X_test, y_test)],
    # early_stopping_rounds=10  <-- remove this line
)

y_pred = model.predict(X_test)

print(f"\n✅ Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=[str(cls) for cls in y_le.classes_]))


Recommendation System — Collaborative Filtering



In [ ]:
# Uninstall and reinstall numpy and surprise to ensure compatibility
!pip uninstall numpy -y
!pip uninstall surprise -y
!pip install numpy surprise pandas scikit-learn seaborn matplotlib xgboost lightgbm imbalanced-learn

In [ ]:
!pip uninstall numpy
!pip install numpy


In [ ]:
!pip uninstall scikit-surprise
!pip install scikit-surprise


In [ ]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall scikit-surprise


In [ ]:
!pip install implicit


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import implicit

# Assuming df has columns: UserId, AttractionId, Rating
# Convert UserId and AttractionId to categorical codes for matrix indexing
df['user_idx'] = df['UserId'].astype('category').cat.codes
df['item_idx'] = df['AttractionId'].astype('category').cat.codes

# Create sparse user-item matrix
user_item_matrix = coo_matrix(
    (df['Rating'].astype(float), (df['user_idx'], df['item_idx']))
)

# Initialize the implicit ALS model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)

# Train the model (implicit library expects item-user matrix)
model.fit(user_item_matrix.T)

# Get recommendations for a user (e.g. user index 0)
user_id = 0
recommended = model.recommend(user_id, user_item_matrix.tocsr(), N=10)

print("Top 10 recommended items for user:", recommended)
